<a href="https://colab.research.google.com/github/RichardFreedman/CRIM-Project-RF/blob/master/CRIM_Intervals_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/HCDigitalScholarship/intervals.git
!pip install httpx


Cloning into 'intervals'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 159 (delta 75), reused 105 (delta 35), pack-reused 0
Receiving objects: 100% (159/159), 63.54 KiB | 2.54 MiB/s, done.
Resolving deltas: 100% (75/75), done.
     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 


In [2]:
from intervals.main_objs import *
import pandas as pd
import ast
import matplotlib
from itertools import tee, combinations
import numpy as np
from fractions import Fraction

In [3]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei', 'https://crimproject.org/mei/CRIM_Mass_0005_5.mei'])


Requesting file from https://crimproject.org/mei/CRIM_Model_0008.mei...
Successfully imported.
Requesting file from https://crimproject.org/mei/CRIM_Mass_0005_5.mei...
Successfully imported.


In [25]:
model, mass = corpus.scores

In [26]:
mel = model.getMelodic()
mel.fillna(value= "-", inplace=True)
mel.head(50)

,[Superius],Altus,Tenor,Bassus
0.0,-,Rest,Rest,Rest
4.0,P4,-,-,-
8.0,-,Rest,Rest,Rest
12.0,P1,-,-,-
16.0,M2,-,Rest,Rest
20.0,M2,P4,-,-
24.0,-M3,-,Rest,Rest
28.0,-,P1,-,-
32.0,Rest,M2,-,Rest
36.0,-,M2,P4,-


In [15]:
ngram_model = model.getNgrams(df=mel, n=4)
ngram_model.head(50)

,[Superius],Altus,Tenor,Bassus
4.0,"P4, P1, M2, M2",NaN,NaN,NaN
12.0,"P1, M2, M2, -M3",NaN,NaN,NaN
20.0,NaN,"P4, P1, M2, M2",NaN,NaN
28.0,NaN,"P1, M2, M2, -M3",NaN,NaN
36.0,NaN,NaN,"P4, P1, M2, M2",NaN
44.0,NaN,NaN,"P1, M2, M2, -M3",NaN
52.0,NaN,NaN,NaN,"P4, P1, M2, M2"
60.0,NaN,NaN,NaN,"P1, M2, M2, -M3"
62.0,"-m2, -M2, -M2, M2",NaN,NaN,NaN
64.0,"-M2, -M2, M2, -M2",NaN,NaN,NaN
